# Quantum Bits

## Qubit

The fundamental unit of classical information storage, processing, and transmission is the bit. Analogously, we define its quantum counterpart, a quantum bit or simply the qubit. Below we define a qubit in CUDA-Q. 

In [1]:
import cudaq

# Build a cudaq kernel.


@cudaq.kernel
def kernel():
    # Allocate a single qubit to the kernel.
    qubit = cudaq.qubit()

Classcial bits are transistor elements whose states can be altered to perform computations. Similarly qubits have physical relizations within superconducting materials, ion-traps, and photonic systems. We shall not concern ourselves with specific qubit architectures but rather think of them as systems which obey the laws of quantum mechanics and linear algebra.

Classical information storage scales linearly if bits have a single state. Access to multiple states, namely a 0 and a 1 allows for information encoding to scale logarithmically. Analagous to the classical states 0 and 1, we use Dirac notation to define a qubit to have the zero state:

$$\ket{0} = \begin{bmatrix} 1 \\ 0 \\ \end{bmatrix}$$

and the one state:

$$\ket{1} = \begin{bmatrix} 0 \\ 1 \\ \end{bmatrix},$$ 

where the symbols $\ket{0}$ and $\ket{1}$ are read as "ket zero"  and "ket one," respectively. 

## Pauli X gate

We can manipulate the state of the qubit via quantum gates. One example of a single-qubit quantum gate is the Pauli-X gate, which can be represented as a matrix:

$$ X =  \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}. $$ 

The Pauli-X gate (or X-gate for short) flips the state of qubit from $\ket{0}$ to $\ket{1}$ through matrix multiplication: 

$$ X \ket{0} = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \\ \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \\ \end{bmatrix}. $$

The CUDA-Q code below creates a quantum kernel that intitializes a qubit in the zero state, applies the X-gate, and measures the state. The kernel is then sampled 1000 times. We will discuss the role of sampling later. 

In [2]:
@cudaq.kernel
def kernel():
    # A single qubit initialized to the zero state, sometimes referred to as the ground state.
    qubit = cudaq.qubit()

    # Apply the Pauli-X gate to the qubit.
    x(qubit)

    # Measurement operator.
    mz(qubit)


# Sample the qubit for 1000 shots to gather statsitics.
result = cudaq.sample(kernel, shots_count=1000)

print(result)

{ 1:1000 }



## Superpositions & Measurements

We have explored two states accessible to us via a qubit. In fact, quantum theory allows one to explore arbitrary linear combinations of states. This is called superpositions: 

$$  \ket{\psi} = \alpha\ket{0} + \beta\ket{1} $$ 

where $\alpha$ and $\beta$ $\in \mathbb{C}$ and $\lvert\alpha\rvert^2+\lvert\beta\rvert^2 =1$. It is important to note that this is still the state of one qubit even though $\ket{\psi}$ is made up of the sum of two kets. 

Quantum theory is probabilistic and hence requires statistical inference to derive observations. Prior to measurement, the state of a qubit is a combination of $\ket{0}$ and $\ket{1}$. Upon measurement, the wavefunction collapse yields either a classical 0 or 1. 

The mathematical theory devised to explain quantum phenomena tells us that the probability of observing the qubit in the state $\ket{0}$ yielding a classical 0 is $\lvert \alpha \rvert ^2$, and the probability of observing the qubit in the state $\ket{1}$ yielding a classical 1 is $\lvert \beta \rvert ^2$. The theory has been verified experimentally countless times and the previous example of sampling the kernel for $X\ket{0}$ demonstrated that $100\%$ of the time when we apply an $X$ gate to the $\ket{0}$ state and measure with respect to the $Z$ basis, we read out a classical 1. Let's look at another example in which our state is in superposition.

The Hadamard gate allows us to put the qubit in an equal superposition state. The Hadamard gate can be represented as a matrix:
$$ H = \tfrac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$$

When $H$ is applied to the $\ket{0}$, the outcome is an equal superposition of $\ket{0}$ and $\ket{1}$, which we give the name the "plus state," denoted by $\ket{+}$:
$$H \ket{0}= \tfrac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \\ \end{bmatrix} = \tfrac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 0 \\ \end{bmatrix} + \tfrac{1}{\sqrt{2}} \begin{bmatrix} 0 \\ 1 \\ \end{bmatrix}\equiv \ket{+}. $$

The probability of finding the qubit in the 0 state is $\lvert \tfrac{1}{\sqrt{2}} \rvert ^2 = \tfrac{1}{2}$, and similarly the probability of finding the qubit in the 1 istate is also $\tfrac{1}{2}$. Lets verify this with some code: 

In [3]:
@cudaq.kernel
def kernel():
    # A single qubit initialized to the zero state.
    qubit = cudaq.qubit()

    # Apply Hadamard gate to the single qubit to put it in equal superposition.
    h(qubit)

    # Measurement operator.
    mz(qubit)


result = cudaq.sample(kernel, shots_count=1000)

print(result)

{ 0:502 1:498 }



Quantum theory is statistical and statistical accuracy increases with sampling. Above we see how with a 1000 shots, the result 0 is yielded roughly 50% of the timem as predicted by the postulate stated above. 

For completeness, the result of applying the Hadamard gate to $\ket{1}$ also yields a state in equal supersposition, which we name the "minus state" and denote by $\ket{-}$.

$$ H \ket{1} =  \tfrac{1}{\sqrt{2}} \ket{0} - \tfrac{1}{\sqrt{2}} \ket{1}  \equiv \ket{-}.$$


## Qubit visualizations


What are the possible states our qubit can be in and how can we build up a visual cue to help us make sense of quantum states and their evolution?

We know our qubit can have 2 distinct states: $\ket{0}$ and $\ket{1}$. Maybe we need a 1 dimensional line whose verticies can represent each of the aforementioned states. 

We also know that a qubit can be in an equal superposition of states: $\ket{+}$ and $\ket{-}$. This now forces us to extend our 1-D line to a 2-D Cartesian coordinate system. 

Later, we will learn of the existence of states that can be represented with $\ket{+i} = i\ket{+}$ and $\ket{-i} = i\ket{-}$; this calls for a 3-D extension. Recall our definition of a quantum state $$  \ket{\psi} = \alpha\ket{0} + \beta\ket{1} $$ 

where $\alpha$ and $\beta$ $\in \mathbb{C}$ and $\lvert\alpha\rvert^2+\lvert\beta\rvert^2 =1.$ We can replace $\alpha$ by $\cos(\theta)$ and $\beta$ by $e^{i\phi}\sin(\theta)$. This 
suggests a 3-D representaiton of all the possible states of a single qubit as a unit sphere, as shown in figure below: 

<img src="images/Bloch_sphere.png" alt="Bloch Sphere" width="300" height="300">



## Gate linearity

Recall that we can manipulate a single qubit with a gate through matrix multiplication. Let's manipulate the single qubits $\ket{0}$ and $\ket{1}$ using the X-gate: 


1. $$  X  \ket{0} = \ket{1}  $$

2. $$  X  \ket{1} = \ket{0}  $$

More generally, for a qubit in a superposition state, quantum gates act linearly because matrix multiplication is linear:

 $$   X (\alpha\ket{0} + \beta\ket{1}) =  X(\alpha\ket{0})+X(\beta \ket{1}) = \alpha X\ket{0}+\beta X\ket{1} =\alpha\ket{1} + \beta\ket{0} $$ 

It is important to note that states such as $\alpha\ket{0} + \beta\ket{1}$ reference a single qubit in a superposition state. Although we have two kets, they both represent a superposition state of one qubit. We shall explore multiple qubits and their notation later. 



## Gate unitarity 

As we evolve quantum states via quantum gates, the normalization condition requires that the sum of modulus squared of amplitudes must equal 1 at all times. That is, for a state 
$  \ket{\psi} = \alpha\ket{0} + \beta\ket{1}$,         $$|\alpha|^2 + |\beta|^2 = 1. $$



This is to adhere to the conservation of probabilities. This requirement translates to a constraint on the types of quantum gates we can define. Up until now, we have seen the $X$ gate and the Hadamard gate.  In general, any unitary operator defines a quantum gate. For a operator $U$ to be unitary, the following equation must be satisfied $$U^{\dagger}U = U^{*^{T}}U = \mathbb{I},$$
where $U^\dagger$ is the conjugate transpose of the matrix $U$. 



## Single-qubit gates

Below we summarize a few single-qubit gates and their effects on quantum states:


$$ X \equiv \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}, \hspace{1cm} X(\alpha\ket{0} + \beta\ket{1}) = \alpha\ket{1} + \beta\ket{0} \hspace{1cm}  $$

$$ Z \equiv \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}, \hspace{1cm} Z(\alpha\ket{0} + \beta\ket{1}) = \alpha\ket{0} - \beta\ket{1}  $$

$$ H \equiv \tfrac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}, \hspace{1cm} H(\alpha\ket{0} + \beta\ket{1}) = \alpha\tfrac{\ket{0}+\ket{1}}{\sqrt{2}} + \beta\tfrac{\ket{0}-\ket{1}}{\sqrt{2}}  $$



